# Oracle: Knowledge Discovery over Databases in Natural Language #
### Johns Hopkins University ###
### Author: Srihari Mohan ###

In [4]:
# Import relevant modules
# nltk for natural language processing tools
# plotly for automated visualizations

import os
import config
import numpy as np
from scipy import stats
import pandas as pd
import nltk
from collections import defaultdict
import re
import itertools
import qgrid
import jellyfish
import plotly
import plotly.plotly as py
import plotly.figure_factory as ff
from plotly.graph_objs import *
from sklearn import svm, linear_model
from sklearn.externals import joblib
from pymining import itemmining, assocrules
import random
from functools import reduce

In [5]:
def is_actor(tag):
    
    '''Return part of speech tags that map to noun phrases in the query'''
    
    return tag in ['NN', 'NNP', 'NNS', 'PRP', 'CD']


def is_desc(tag):
    
    '''Return part of speech tags that map to modifiers in the query'''
    
    return tag in ['CD', 'JJ', 'JJR', 'JJS', 'RBR', 'RBS', 'VBG']


def is_prep(tag):
    
    '''Return part of speech tags that map to prepositions in the query'''
    
    return tag == 'IN'


def is_verb(tag):
    
    '''Return part of speech tags that map to verb phrases in the query'''
    
    return re.match(r'VB.*', tag[1]) is not None and tag[0] not in config.stoplist['V']


def is_genitive(tag):
    
    '''Return part of speech tags that map to possessive modifiers in the query'''
    
    return tag[0] == '->' or tag[1] in ['POS', 'PRP$']


def is_adv(tag):
    
    '''Return part of speech tags that map to adverbs in the query'''
    
    return tag == 'RB'


def is_gerund(tag):
    
    '''Return part of speech tags that map to gerunds in the query'''
    
    return tag == 'VBG'


def is_conj(tag):
    
    '''Return part of speech tags that map to conjunctions in the query'''
    
    return tag in ['and', 'or', 'not']


def substitute_entity(subs, ent):
    
    '''Return if the phrase entity passed is a built-in function in dictionary subs'''
    
    return subs[ent]

In [6]:
# initialize filters set to contain all keywords that are not conjunctions
for f_key in config.keywords:
    if config.keywords[f_key] != '->' and not is_conj(f_key):
        config.filters.add(config.keywords[f_key])

In [4]:
# display interactive data view
qgrid.show_grid(config.X, show_toolbar=True, grid_options={'forceFitColumns': False, 'defaultColumnsWidth': 200})

In [7]:
def user_input(args):
    
    '''Prompt user to clarify ambiguous tokens in the query'''
    
    print('Within the context of your data, what do you mean by ' + "\'" + args + "\'")
    
    feature = input('Relevant feature: ') # prompt for feature
    value = input('Associated value: ') # prompt for value
    
    return (feature, [value])


def identify_val_via_user(relevant, value):
    
    '''Given a feature determined to be relevant, prompt user for its associated value if ambiguous'''
    
    print('Within the context of your data, how does ' \
          + "\'" + value + "\'" + 'relate to the field ' + "\'" + relevant + "\'")
    val = input('Relevant value: ') # prompt for value
    
    return [val]

In [8]:
def sequential_entity(e_list, b_feat, actors):
    
    '''Perform sequence of filter reductions on the entity list'''
    
    response = str(' '.join([re.findall(r'^\(?(.+[^)])\)?', actor[0])[0] for actor in actors])).title()
    criteria, added, expr = generate_criteria(e_list) # generate entity criteria (feature, [values])
    
    for i in range(len(criteria)): # sequentially filter database on entity criteria
        
        perform_filter('filter', criteria[i], b_feat) # perform filter 
        feat = criteria[i][0]
        config.MOST_RECENT_QUERY.add(feat) # cache this query as most recent
        config.ITEMSETS[len(config.ITEMSETS)-1].add(feat) # add to frequent itemsets cache
        config.EXP_DECAY[feat] = 1 # set distance of feature since last fetched to 1
        
        for j in range(i, len(criteria)):
            config.COOCCURENCE_HASH[feat][criteria[j][0]] += 1 # update cooccurrence hash
            
    for e in expr:
        result, plot, status = match_expr(e, b_feat, response) # sequentially execute all user defined actions
        
    for feat in added: # restore original columns
        del config.filtered[feat]
        del config.DOMAIN_KNOWLEDGE[feat]
        
    config.ITEMSETS[len(config.ITEMSETS)-1] = tuple(config.ITEMSETS[len(config.ITEMSETS)-1])
    
    if status: # output textual response
        
        response = "Oracle\'s response: " + str(result)
        printmd('**\n' + response + '**')
        rterms = generate_features_rf_()
        
        if rterms:
            printmd('**\n' + '(Relevance Feedback) Investigate Features More Like This: ' + str(rterms) + '**\n')
    
    if plot is not None:
        return plot # return automated visualization


def generate_criteria(e_list):
    
    '''Parse each entity token in the query into list of (feature, [value])'''
    
    entities = []
    genitive = False
    
    for i in range(len(e_list)): # iterate over entity list
        
        if e_list[i] == config.GENITIVE:
            genitive = True
        elif '=>*' not in e_list[i] and not genitive:
            entities.append(re.findall(r'^\(?(.+[^)])\)?', e_list[i])[0].split()) # extract each entity
        else:
            entities.append(re.findall(r'^\(?(.+[^)])\)?', e_list[i]))
            genitive = False
            
    entities = list(itertools.chain.from_iterable(entities))
    prev_feat, genitive = None, False
    criteria, added, expr = [], [], []
    rec_item = config.RECOMMENDATION.data[config.qry] # hash current query into RECOMMENDATIONS hash
    conj = None
    
    for entity in entities:
        
        if is_expr(entity): # check if token is a pre-built expression
            expr.append(entity)
            continue
        elif is_conj(entity): # check if token is a conjunction and tag it
            conj = entity
            continue
        
        curr_feat = match_(entity) # extract relevant feature
        start = config.qry.find(entity)
        end = start + len(entity)
        rec_item.index_hash[curr_feat] = (start, end) 
        config.RECOMMENDATION.data[config.qry] = rec_item
        
        if curr_feat is None:
            return user_input(entity) # prompt user to clarify input
        else:
            
            val = match_arg_to_feature_value(curr_feat, entity) # extract associated value to relevant feature
                
            while val is None:
                val = identify_val_via_user(curr_feat, val) # prompt user if argument specification is ambiguous
                
            if criteria: # handle conjunctions in query
                if conj == 'OR': # union logic
                    if criteria[len(criteria)-1][1] == curr_feat:
                        criteria[len(criteria)-1][1][0] += '+' + '+'.join(val)
                    else:
                        pass
                elif conj == 'NOT': # negation logic
                    criteria.extend([(curr_feat, ['!' + v_key]) for v_key in val])
                else:
                    criteria.append((curr_feat, ['+'.join(val)]))
            else: # intersection logic
                criteria.append((curr_feat, ['+'.join(val)]))
            conj = None
                
    features = [feat for feat in config.RECOMMENDATION.data[config.qry].index_hash]
    features.sort() # sort features to specify common key
    config.RECOMMENDATIONS[str(features)].append(config.RECOMMENDATION) # hash features in RECOMMENDATION
    
    return criteria, added, expr


def is_expr(entity):
    
    '''Holds suite of pre built functions (averages, etc.)'''
    
    return entity in {'strike rate', 'on-base percentage', 'average ball velocity', 'average pitch velocity',
                     'average x location', 'average pitch height', 'average x acceleration', 'average z acceleration'}


def calc_score_(entity, group):
    
    '''Return filtered batched result after executing pre-built function '''
    
    if entity == 'strike rate':
        calc = group[(group['type'] == 'S') | (group['type'] == 'X')].shape[0] / group.shape[0]
    elif entity == 'on-base percentage':
        calc = group[((group['event'] == 'Single') | (group['event'] == 'Double') | (group['event'] == 'Triple') | 
                      (group['event'] == 'Home Run') | (group['event'] == 'Walk') | 
                      (group['event'] == 'Field Error'))].shape[0] / group.shape[0]
    elif entity == 'average ball velocity' or entity == 'average pitch velocity':
        calc = group['start_speed'].sum() / group.shape[0]
    elif entity == 'average x location':
        calc = group['px'].sum() / group.shape[0]
    elif entity == 'average pitch height':
        calc = group['pz'].sum() / group.shape[0]
    elif entity == 'average x acceleration':
        calc = group['ax'].sum() / group.shape[0]
    elif entity == 'average z acceleration':
        calc = group['az'].sum() / group.shape[0]
        
    return calc if calc is not None else None


def match_expr(entity, b_list, title):
    
    '''driver for filtering on pre-built functions'''
    
    plot, status = None, True
    result, names, plot_l = '', [], []
    
    if b_list is not None: # if b_list is not empty, perform a plot over distribution of feature
        
        for name, group in config.filtered: # iterate over each group
            calc = calc_score_(entity, group)
            if calc is not None:
                names.append(name) # add to x labels list
                plot_l.append(calc) # add to y values list
                result += (name + ': ' + str(calc) + '\n')
        
        if plot_l: # if calculated results are returned, generate the plot
            plot = automate_plot_by_(names, plot_l, entity, title, 'bar') # call plotting subroutine
            status = False # set time_series plot status to False
    
    else: # otherwise plot a timeseries by default
        result = { # evaluate expression over filtered dataset and store result
            'strike rate': lambda Z: str(Z[(Z['type'] == 'S') | (Z['type'] == 'X')].shape[0] / Z.shape[0]) if Z.shape[0] > 0 else 0,
            'on-base percentage': lambda Z: str(Z[((Z['event'] == 'Single') | (Z['event'] == 'Double') | (Z['event'] == 'Triple') | 
                                                   (Z['event'] == 'Home Run') | (Z['event'] == 'Walk') | 
                                                   (Z['event'] == 'Field Error'))].shape[0] / Z.shape[0]) if Z.shape[0] > 0 else 0,
            'average ball velocity': lambda Z: str(Z['start_speed'].sum() / Z.shape[0]) if Z.shape[0] > 0 else 0,
            'average pitch velocity': lambda Z: str(Z['start_speed'].sum() / Z.shape[0]) if Z.shape[0] > 0 else 0,
            'average x location': lambda Z: str(Z['px'].sum() / Z.shape[0]) if Z.shape[0] > 0 else 0,
            'average pitch height': lambda Z: str(Z['pz'].sum() / Z.shape[0]) if Z.shape[0] > 0 else 0,
            'average x acceleration': lambda Z: str(Z['ax'].sum() / Z.shape[0]) if Z.shape[0] > 0 else 0,
            'average z acceleration': lambda Z: str(Z['az'].sum() / Z.shape[0]) if Z.shape[0] > 0 else 0
        }[entity](config.filtered)
        
        time_series = config.filtered.groupby('Date') # group filtered dataset by date
        
        for name, group in time_series:
            calc = calc_score_(entity, group) # calculate result
            if calc is not None:
                names.append(name)
                plot_l.append(calc)
                
        if plot_l:
            plot = automate_plot_by_(names, plot_l, entity, title, 'scatter') # plot time series chart
            
    return result, plot, status


def is_number(s):
    
    '''Return if a string can be parsed as a number'''
    
    try:
        float(s)
        return True
    except ValueError:
        return False

In [27]:
def sequential_filter(f_list):
    
    '''Execute sequential filters over dataset on supplied filtering criteria'''
    
    config.MOST_RECENT_QUERY = set() # reinitialize MOST_RECENT_QUERY
    config.ITEMSETS.append(set()) # reinitialize ITEMSETS
    
    for feat in config.EXP_DECAY:
        config.EXP_DECAY[feat] += 1 # increment all features distance from being last fetched by 1
        
    config.filtered = config.X.copy() # reinitialize filtered
    filters = feature_assoc_filters_helper(f_list) # generate filtering criteria as list of (feature, [values])
    grouped, b_list = None, []
    
    for i in range(len(filters)): # iterate over each filtering criterion
        
        f_, c_ = filters[i][0], filters[i][1] # extract filtering operation, criteria
        feat = c_[0]
        config.MOST_RECENT_QUERY.add(feat)
        config.ITEMSETS[len(config.ITEMSETS)-1].add(feat) # update ITEMSETS
        config.EXP_DECAY[feat] = 1 # set distance since being fetched to 1
        
        for j in range(i, len(filters)):
            config.COOCCURENCE_HASH[feat][filters[j][1][0]] += 1 # update cooccurence hash
            
        if not c_[1]:
            b_list.append(feat) # add to b_list if filtering criteria is over a feature's entire distribution
            continue
            
        perform_filter(f_, c_, False) # perform appropriate filter
        
    if len(b_list) > 0:
        grouped, b_feat = by_([b_list, []])
        
    return (config.filtered, None) if grouped is None else (grouped, b_feat)


def feature_assoc_filters_helper(filters):
    
    '''Return list of (relevant feature, [associated values]) tuples to filter automatically'''
    
    f_ = [] # initilize result list
    config.RECOMMENDATION = config.Recommendation(config.qry) # initialize RECOMMENDATION to hash on current query
    rec_item = config.RecommendationItem()
    prev_numeric = None
    
    for f_item in filters:
        
        tokens = re.findall(r'\(.+?\)|AND|OR|NOT', f_item) # tokenize filters
        tokens[0] = re.findall(r'%(.+)%', tokens[0])[0]
        tokens[1] = tokens[1][1:]
        tokens = [re.findall(r'\((.+?)\)', token)[0] if '(' in token else token for token in tokens]
        prev_feat, relevant, val, cat = None, None, None, ''
        i, inc, c_flag = 1, False, False
        
        while i < len(tokens): # iterate over each filter in the filtered list
            
            if prev_numeric is not None:
                relevant = prev_numeric
                val = [tokens[i]] # set argument value to match token if a numeric feature
                
            if relevant is None:
                
                relevant = match_(tokens[i]) # extract relevant feature for tokens[i]
                if relevant is None:
                    relevant, val = user_input(tokens[i]) # prompt user if ambiguous
                start = config.qry.find(tokens[i]) # start index stored before cached in RECOMMENDATION
                end = start + len(tokens[i]) # end index stored before cached in RECOMMENDATION
                rec_item.index_hash[relevant] = (start, end) 
                
                # set current recommendation item as the value to RECOMMENDATION hashsed on the current query
                config.RECOMMENDATION.data[config.qry] = rec_item 
                
            if val is None:
                
                cat += tokens[i]
                
                while (i+1) < len(tokens) and is_conj(tokens[i+1].lower()): # check if next token is a conjunction
                    
                    if (i+2) < len(tokens):
                        
                        inc = True
                        next_feat = match_(tokens[i+2]) # extract relevant feature for this next token
                        
                        if next_feat is None:
                            next_feat, _ = user_input(tokens[i+2]) # if ambiguous prompt the user
                        
                        if relevant != next_feat: # check if the current parsed feature matches the next feature
                            relevant = next_feat # if not append to the result list sequentially
                            i += 2
                            break
                        
                        else:
                            
                            # otherwise append the next feature value to the running result
                            cat += tokens[i+1] + tokens[i+2] 
                            i += 3
                            
                for core_entity in config.CORE:
                    if jellyfish.jaro_distance(core_entity, cat) > config.NAME_THRESHOLD:
                        c_flag = True
                        break
                        
            if not c_flag:
                
                if val is None:
                    
                    # call subroutine to generate filter criteria for current (feature, [arguments]) pair
                    result = generate_filter_criteria(cat) 
                    
                    if not result[1] or result[1][0] != 'is.numeric':
                        f_.append((tokens[0], result)) # append to result list
                        prev_numeric = None
                    else:
                        prev_numeric = result[0]
                else:
                    f_.append((tokens[0], (relevant, val))) # append to result list
                    prev_numeric = None
                    
            if not inc: 
                i += 1
                
            inc, c_flag, val, cat = False, False, None, ''
            
    return f_
        

def perform_filter(f_key, c_key, b_list):
    
    '''Perform relevant filter'''
    
    result = {
        'filter': lambda D: filter_(c_key, b_list),
        'by': lambda D: by_(c_key),
        'over': lambda D: over_(c_key),
        'under': lambda D: under_(c_key),
        'between': lambda D: between_(c_key),
        'except': lambda D: except_(c_key),
        'near': lambda D: near_(c_key),
        'until': lambda D: until_(c_key),
        'to': lambda D: to_(c_key),
        'after': lambda D: after_(c_key),
        'before': lambda D: before_(c_key),
        'against': lambda D: compare_(c_key)
    }[f_key](config.filtered)
    

def generate_filter_criteria(args, hint=None):
    
    '''Return relevant (feature, [values]) tuple that matches argument'''
    
    criteria = []
    
    for identifier in config.IDENTIFIERS:
        for name in config.IDENTIFIERS[identifier]:
            if jellyfish.jaro_distance(args, name) > config.NAME_THRESHOLD:
                return (identifier, name)
            
    c_keys = ['AND', 'OR', 'NOT']
    pat = '((?:' + '|'.join(c_keys) + '))'
    c_list = re.split(pat, args) # split on conjunctions
    
    if c_list:
        relevant = match_(c_list[0]) # extract relevant feature
        
    if not c_list or relevant is None:
        return user_input(args) # if ambiguous, prompt user to specify
    
    print('\nfeature association: most relevant feature is ' + relevant)
    
    if 'is.numeric' in list(config.DOMAIN_KNOWLEDGE[relevant].values())[0]:
        return (relevant, ['is.numeric']) # tag feature with 'is.numeric' if it is numeric
    elif args in config.DOMAIN_KNOWLEDGE[relevant]:
        return (relevant, [])
    
    conj = None
    
    for term in c_list:
        
        if is_conj(term.lower()):
            conj = term
        else:
            lookup = re.match('(.+)', term).group() # extract token
            val = match_arg_to_feature_value(relevant, lookup) # look up associated filter criteria on feature
            print('generate_criteria: ' + str(val))
            
            while val is None:
                val = identify_val_via_user(relevant, val) # prompt user if associated value is ambiguous
                
            if criteria:
                if conj == 'OR' or len(val) > 1: # handle union logic
                    criteria[len(criteria)-1] += '+' + '+'.join(val)
                elif conj == 'NOT': # handle negation logic
                    criteria.extend(['!' + v_key for v_key in val])
                else:
                    criteria.append('+'.join(val))
            else: # handle intersection logic
                criteria.append('+'.join(val))
                
            conj = None
            
    return (relevant, criteria)
    

def filter_(c_key, b_list):
    
    '''Perform filter on categorical feature'''
    
    feature = c_key[0] # extract feature
    args = c_key[1] # extract relevant values of feature to filter on
    print('(' + str(feature) + ', ' + str(args) + '): filter_')
    
    if b_list: # handle automated filtering over distribution of a feature
        for index, f_tok in enumerate(args): # iterate over each argument
            
            f_ = re.split(r'(?:\+|!)', f_tok) # split by filters handled with conjunctive logic
            
            if '!' in f_tok: # handle negation
                exec_str = "config.filtered = config.filtered.apply(lambda g: (g[g[\'" + feature + "\'] != \'" + f_[1] + "\']))"
                print(exec_str) # generate string to execute code that automates filtering step
                exec(exec_str) # execute code
            else:
                union = [item for item in f_]
                exec_str = "config.filtered = config.filtered.apply(lambda g: (g[g[\'" + feature + "\'].isin(" + str(union) + ")]))"
                print(exec_str) # generate string to execute code that automates filtering step
                exec(exec_str) # execute code
                
            # filter over each value of a categorical feature
            config.filtered = config.filtered.groupby(b_list) # group by features listed in b_list
    else:
        exec_str = 'config.filtered['
        
        for index, f_tok in enumerate(args): # iterate over each argument
            
            f_ = re.split(r'(?:\+|!)', f_tok) # split on conjunctions
            
            if '!' in f_tok: # negation logic
                exec_str = "config.filtered = config.filtered[(config.filtered[\'" + feature + "\'] != \'" + f_[1] + "\')"
                print('\n' + exec_str)
                exec(exec_str) # execute automated filter
                continue
                
            exec_str = "config.filtered[(config.filtered[\'" + feature + "\'] == \'" + f_[0] + "\')"
            
            for i in range(1, len(f_)):
                exec_str += " | (config.filtered[\'" + feature + "\'] == \'" + f_[i] + "\')" # handle union logic
                
            exec_str += ']'
            exec_str = "config.filtered = " + exec_str
            print('\n' + exec_str)
            exec(exec_str) # execute automated filter
    
    # return filtered result
    return config.filtered


def by_(c_key):
    
    '''Filter database over distribution of a feature'''
    
    features = c_key[0] # extract features
    config.filtered = config.X.groupby(features) # group by features
    
    return config.filtered, features


def over_(c_key):
    
    '''Filter database on values over a threshold for a feature'''
    
    feature = c_key[0] # extract feature
    args = c_key[1] # extract value to filter on
    exec_str = "config.filtered = config.filtered[(config.filtered[\'" + feature + "\'] > " + args[0] + ")]"
    print('\n' + exec_str) 
    exec(exec_str) # execute automated filter
    
    return config.filtered


def under_(c_key):
    
    '''Filter database on values under a threshold for a feature'''
    
    feature = c_key[0] # extract feature
    args = c_key[1] # extract value to filter on
    exec_str = "config.filtered = config.filtered[(config.filtered[\'" + feature + "\'] < " + args[0] + ")]"
    print('\n' + exec_str)
    exec(exec_str) # execute automated filter
    
    return config.filtered


def between_(c_key):
    
    '''Filter database on values between two thresholds for a feature'''
    
    feature = c_key[0] # extract feature
    args = c_key[1] # extract value to filter on
    left, right = args[0], args[1]
    exec_str = ("config.filtered = config.filtered[(config.filtered[\'" + feature + "\'] > " + left + ")"
                "& (config.filtered[\'" + feature + "\'] < " + right + ")]")
    print('\n' + exec_str)
    exec(exec_str) # execute automated filter
    
    return config.filtered


def except_(c_key):
    
    '''Filter database on values of a categorical feature except for those specified in the argument'''
    
    feature = c_key[0] # extract feature
    args = c_key[1] # extract values to filter on
    unique_vals = list(set(list(config.filtered[feature].unique())) - set([args]))
    unique_vals = [x for x in unique_vals if x == x]
    unique_vals = '+'.join(unique_vals)
    config.filtered = filter_([feature, unique_vals], None, config.filtered) # perform filter
    
    return config.filtered


def near_(c_key):
    
    '''Filter database on values within +/- 0.5 std of the argument on a numerical feature'''
    
    feature = c_key[0] # extract relevant feature
    args = c_key[1] # extract values to filter on
    left = str(float(args[0]) - 0.5 * config.filtered[feature].std()) # left bound -0.5 std
    right = str(float(args[1]) + 0.5 * config.filtered[feature].std()) # right bound +0.5 std
    
    if left > right:
        left, right = right, left
        
    config.filtered = between_([feature, [left,right]]) # perform filter
    
    return config.filtered


def automate_plot_by_(x, y, entity, title, chart_type):
    
    '''Return automated visualization relevant to queried features'''
    
    font=dict(family='Courier New, monospace', size=24, color='#7f7f7f') # set font
    
    xaxis=dict( # set x-axis plot attributes
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
    
    yaxis=dict( # set y-axis plot attributes
        title = entity,
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
    
    if chart_type == 'bar':
        data = [Bar(x=x, y=y)] # set data for bar chart chart
    else:
        data = [plotly.graph_objs.Scatter(x=x, y=y)] # set data for defaulted time series scatter
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label='1m',
                     step='month',
                     stepmode='backward'),
                dict(count=6,
                     label='6m',
                     step='month',
                     stepmode='backward'),
                dict(step='all')
            ])
        )
        
        xaxis['rangeselector'] = rangeselector # add range selector
        xaxis['rangeslider'] = dict()
        
    layout = plotly.graph_objs.Layout( # adjust plot layout
        title=title,
        font=font,
        xaxis=xaxis,
        yaxis=yaxis
    )
    
    fig = plotly.graph_objs.Figure(data=data, layout=layout) # store automated plot
    
    return py.iplot(fig, filename='extract_bar') if chart_type == 'bar' else py.iplot(fig, filename='extract_scatter')        
    

def match_(args):
    
    '''Return relevant feature to filter on'''
    
    max_conf, max_feat = 0, ''
    
    for entry in config.DOMAIN_KNOWLEDGE: # iterate over terms in the system's domain knowledge
        
        if args in config.DOMAIN_KNOWLEDGE[entry]: # if term matches exactly return it
            return entry
        
        tokens = args.split() # otherwise split tokens and accumulate evidence of belonging to each feature
        conf_f = 0 # confidence
        
        for token in tokens: # iterate over each token
            
            curr = 0
            
            for desc in list(config.DOMAIN_KNOWLEDGE[entry].keys()): # iterate over each term in domain knowledge
                
                # compute string similarity of query vs term in DOMAIN_KNOWLEDGE
                curr = max(jellyfish.jaro_distance(desc, token), curr) # string similarity by jaro_distance
            
            conf_f += curr # accumulate confidence score for feture
            
        conf_f /= len(tokens) # scaled confidence level of feature match
        
        if conf_f > max_conf: # update max condfidence level and associated feature
            max_conf, max_feat = conf_f, entry
    
    if max_conf > config.CONF_THRESHOLD:
        return max_feat # return result
    else:
        return None


def match_arg_to_feature_value(feature, args):
    
    '''Return list of relevant arguments that match feature value'''
    
    print('(' + feature + ', ' + args + '): ' + 'match_arg_to_feature_value')
    
    if 'is.numeric' in list(config.DOMAIN_KNOWLEDGE[feature].values())[0]:
        if not is_number(args):
            return None
        return [args] # return argument if relevant feature is tagged as being numeric
    
    unique_vals = list(set(list(itertools.chain.from_iterable(config.DOMAIN_KNOWLEDGE[feature].values()))))
    vals = {key:0 for key in unique_vals} # initialize hash to accumulate evidence for each value of feature
    tokens = args.split() # tokenize argument
    max_val, max_arg = 0, []
    
    for token in tokens: # iterate over each token
        
        for lookup in config.DOMAIN_KNOWLEDGE[feature]: # lookup relevant modifiers in domain knowledge
            
            jaro = jellyfish.jaro_distance(token, lookup) # compute string similarity of modifier vs query token
            
            # jaro normalized as a confidence between [0, 1]
            if jaro > config.CONF_THRESHOLD: # check if confidence is greater than preset threshold
                
                # iterate over list of feature values that match the current modifier in domain knowledge
                for i in range(len(config.DOMAIN_KNOWLEDGE[feature][lookup])):
                    
                    vals[config.DOMAIN_KNOWLEDGE[feature][lookup][i]] += jaro # accumulate evidence for lookup in hash
                    
                    if vals[config.DOMAIN_KNOWLEDGE[feature][lookup][i]] > max_val:
                        
                        max_val = vals[config.DOMAIN_KNOWLEDGE[feature][lookup][i]] # update max confidence
                        max_arg = [config.DOMAIN_KNOWLEDGE[feature][lookup][i]] # update associated max argument
                    
                    elif vals[config.DOMAIN_KNOWLEDGE[feature][lookup][i]] == max_val:
                        
                        # accomodate for series of feature values that match the current modifier equally 
                        max_arg.append(config.DOMAIN_KNOWLEDGE[feature][lookup][i])
                        
    return max_arg if max_arg else None

In [10]:
def generate_features_rf_():
    
    '''Generate Bag-of-words of Relevant Features on Most Recent Query
    
    Current implementation supports suggestion of relevant features by relevance feedback (Rochio algorithm)
    Current implementation also supports suggestion of relevant features by frequent itemsets
    Defaulted to implementation of relevance feedback, with results cached for future reference
    
    Oracle caches features fetched over time and recalculates their weights in the cooccurence hash by
    time since last hit using an exponential decay
    '''
    
    qry_vector = defaultdict(float)
    rterms, nrterms = [], []
    
    # cooccurence_hash => 'document vector'
    # represents (term, features) matrix where weight for each (term, feature) is dependent on cooccurence strength
    cooccurence_hash = config.COOCCURENCE_HASH.copy()
    
    for term, steps in config.EXP_DECAY.items():
        cooccurence_hash[term][term] *= config.DECAY * (np.e ** (-config.DECAY * steps)) # update weights by exp decay
        
    for feat, term_wgts in cooccurence_hash.items(): # iterate over feature, weight pairs in cooccurence hash
        
        if feat in config.MOST_RECENT_QUERY: # construct query vector on terms in most recent query
            qry_vector[feat] += 1
            rterms.append(term_wgts)
        else:
            nrterms.append(term_wgts) # construct list of nonrelevant terms for Rochio
            
    reform = rochio_algo(qry_vector, rterms, nrterms, 1, 0.75, 0.15) # compute reformulated query vector by Rochio
    rterms = [] # reinitialize rterms to hold suggested terms to investigate
    
    for term in cooccurence_hash: # iterate over each term (key) in cooccurence hash
        cos_sim = cosine_sim(reform, cooccurence_hash[term]) # compute similarity of each vector in cooccurence_hash
        rterms.append((term, cos_sim)) # append to rterms
        
    rterms.sort(key=lambda x: x[1]) # sort rterms before inserting into cache
    
    # only include features deemed relevant over a preset threshold
    rterms = [term[0] for term in rterms if term[1] > config.RELEVANCE_FEEDBACK_THRESHOLD]
    
    return rterms

def generate_queries_itemsets():
    
    '''Return association rules from frequent itemsets analysis of relevant features to investigate'''
    
    relim_input = itemmining.get_relim_input(config.ITEMSETS)
    item_sets = itemmining.relim(relim_input, min_support=config.ASSOC_MIN_SUPPORT) # generate frequent itemsets
    rules = assocrules.mine_assoc_rules(item_sets, min_support=config.ASSOC_MIN_SUPPORT, 
                                        min_confidence=config.ASSOC_MIN_CONFIDENCE) # generate association rules
    rules.sort(key=lambda x: -1 * x[2] * x[3]) # sort rules before inserting into cache
    
    return rules

In [11]:
def rochio_algo(qry_vector, rel_terms, nonrel_terms, a, B, y):
    
    '''Relevance Feedback by Rochio Algorithm for Automated Query Suggestion
    
    Extension of original algorithm by caching results for future reference
    '''
    
    rels, nonrels = defaultdict(float), defaultdict(float)
    dr, dnr = len(rel_terms), len(nonrel_terms)
    
    for rel_term in rel_terms: # iterate over relevant feature set
        
        if dr <= 0: break
            
        for rel_key in rel_term: # iterate over each (feature, weight) tuple in rels
            rels[rel_key] += (B / dr) * rel_term[rel_key] # reweight each feature weight in relevant set
            
    for nonrel_term in nonrel_terms: # iterate over nonrelevant feature set
        
        if dnr <= 0: break
            
        for nonrel_key in nonrel_term: # iterate over each (feature, weight) tuple in nonrels
            nonrels[nonrel_key] += (y / dnr) * nonrel_term[nonrel_key] # reweight each feature weight in nonrelevant set 
            
    for term in qry_vector:
        qry_vector[term] *= a # rewight initial query vector by alpha
    
    reform = defaultdict(float) # initialize reformulated query vector
    
    for text in [qry_vector, rels]:
        for term in text:
            reform[term] += text[term]
            
    for nonrel_key in nonrels:
        
        if nonrel_key in reform:
            
            reform[nonrel_key] -= nonrels[nonrel_key] # offset reformulated query to drift from the nonrelevant set
            
            if reform[nonrel_key] < 0:
                reform.pop(nonrel_key, None) # reset features with weights < 0 to 0 in reform
    
    # return reformulated query vector
    return reform

def cosine_sim(vec1, vec2, vec1_norm = 0.0, vec2_norm = 0.0):
    
    '''Return cosine similarity between two vectors'''
    
    if not vec1_norm:
        vec1_norm = sum(v * v for v in vec1.values())
    if not vec2_norm:
        vec2_norm = sum(v * v for v in vec2.values())

    # save some time of iterating over the shorter vec
    if len(vec1) > len(vec2):
        vec1, vec2 = vec2, vec1

    # calculate the inner product
    inner_product = sum(vec1.get(term, 0) * vec2.get(term, 0) for term in vec1.keys())

    return inner_product / np.sqrt(vec1_norm * vec2_norm)

In [20]:
from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

# request query from user
config.qry = input('Query your data: ')

Query your data: what is the on-base percentage of lefty batters against oriole pitchers on fastballs or curveballs where the start velocity of the pitch is over 90, the horizontal movement of the pitch is under 12, height is over 2


In [21]:
tokens = nltk.word_tokenize(config.qry) # tokenize user query
pos_tag = nltk.pos_tag(tokens) # apply part of speech tagger
pos_tag

[('what', 'WP'),
 ('is', 'VBZ'),
 ('the', 'DT'),
 ('on-base', 'JJ'),
 ('percentage', 'NN'),
 ('of', 'IN'),
 ('lefty', 'JJ'),
 ('batters', 'NNS'),
 ('against', 'IN'),
 ('oriole', 'JJ'),
 ('pitchers', 'NNS'),
 ('on', 'IN'),
 ('fastballs', 'NNS'),
 ('or', 'CC'),
 ('curveballs', 'NNS'),
 ('where', 'WRB'),
 ('the', 'DT'),
 ('start', 'NN'),
 ('velocity', 'NN'),
 ('of', 'IN'),
 ('the', 'DT'),
 ('pitch', 'NN'),
 ('is', 'VBZ'),
 ('over', 'IN'),
 ('90', 'CD'),
 (',', ','),
 ('the', 'DT'),
 ('horizontal', 'JJ'),
 ('movement', 'NN'),
 ('of', 'IN'),
 ('the', 'DT'),
 ('pitch', 'NN'),
 ('is', 'VBZ'),
 ('under', 'IN'),
 ('12', 'CD'),
 (',', ','),
 ('height', 'NN'),
 ('is', 'VBZ'),
 ('over', 'IN'),
 ('2', 'CD')]

In [22]:
actors, ind, verbs = [], [], []
prev = False

for i, tag in enumerate(pos_tag): # enumerate over each element of the tagged list
    
    tag = list(tag)
    
    if is_prep(tag[1]) or pos_tag[i][0] in config.keywords: # check if token is a preposition or a keyword
        actors.append(tag)
        prev = False    
    elif is_genitive(tag) or is_verb(tag) or is_actor(tag[1]):
        
        if is_genitive(tag) or is_verb(tag): # check if tagged element is a possessive modifier or a verb phrase
            
            if is_genitive(tag): 
                tag[0] = config.GENITIVE # reset query token to be the genitive placeholder '->'
            elif is_verb(tag):
                verbs.append(i) # add to verbs list
                
            actors.append(tag) # add to actors list
            prev = False
        else:
            
            if not prev:
                actors.append(tag)
                ind.append(i)
            else:
                actors[len(actors)-1][0] += ' ' + tag[0] # concatenate noun phrases with adjacent NNP
            
            prev = True
    else:
        prev = False
        
actors

[['percentage', 'NN'],
 ['of', 'IN'],
 ['batters', 'NNS'],
 ['against', 'IN'],
 ['pitchers', 'NNS'],
 ['on', 'IN'],
 ['fastballs', 'NNS'],
 ['or', 'CC'],
 ['curveballs', 'NNS'],
 ['where', 'WRB'],
 ['start velocity', 'NN'],
 ['of', 'IN'],
 ['pitch', 'NN'],
 ['over', 'IN'],
 ['90', 'CD'],
 ['movement', 'NN'],
 ['of', 'IN'],
 ['pitch', 'NN'],
 ['under', 'IN'],
 ['12', 'CD'],
 ['height', 'NN'],
 ['over', 'IN'],
 ['2', 'CD']]

In [23]:
a_ind, v_ind = 0, 0 # initialize actor index and verb index
prev = False

for i in range(len(ind)):
    
    if a_ind < len(actors) and v_ind < len(verbs) and is_verb(actors[a_ind]):
        
        index = verbs[v_ind] + 1
        
        while index < len(pos_tag) and is_adv(pos_tag[index][1]):
            actors[a_ind][0] += ' ' + pos_tag[index][0] # group adverbs and verb phrases as a single entity
            index += 1
            
        v_ind += 1
        
    while a_ind < len(actors) and not is_actor(actors[a_ind][1]):
        a_ind += 1
    
    index = ind[i]-1
    
    while index >= 0 and is_desc(pos_tag[index][1]):
        
        # concatenate noun phrases with adjacent modifiers
        actors[a_ind][0] = pos_tag[index][0] + ' ' + actors[a_ind][0]
        index -= 1
        
    a_ind += 1
    
for i, tag in enumerate(actors):
    if is_actor(tag[1]):
        tag[0] = '(' + tag[0] + ')' # wrap noun phrases in parenthesis for tagging

# remove gerund and noun phrase modifers adjacent to the concatenated sets contructed above
actors[:] = [actors[i] for i in range(len(actors)) if not ((i+1) < len(actors) and 
                                                       (is_gerund(actors[i][1]) and 
                                                       is_actor(actors[i+1][1])))]        
actors

[['(on-base percentage)', 'NN'],
 ['of', 'IN'],
 ['(lefty batters)', 'NNS'],
 ['against', 'IN'],
 ['(oriole pitchers)', 'NNS'],
 ['on', 'IN'],
 ['(fastballs)', 'NNS'],
 ['or', 'CC'],
 ['(curveballs)', 'NNS'],
 ['where', 'WRB'],
 ['(start velocity)', 'NN'],
 ['of', 'IN'],
 ['(pitch)', 'NN'],
 ['over', 'IN'],
 ['(90)', 'CD'],
 ['(horizontal movement)', 'NN'],
 ['of', 'IN'],
 ['(pitch)', 'NN'],
 ['under', 'IN'],
 ['(12)', 'CD'],
 ['(height)', 'NN'],
 ['over', 'IN'],
 ['(2)', 'CD']]

In [24]:
prev_p, prev_a, prev_f, flag = False, False, False, -1 # set preposition, actor, filter flags
open_v = False # set flag to check if currently parsing a verb phrase
extag = '' # parsed result
pos_index = 0 # holds current position to edit result at in the finite state transduction

for i in range(len(actors)):
    
    if flag == 0:
        
        if actors[i][0].lower() != 'of':
            extag += '=>*(%filter%)(' + actors[i][0]
            flag, open_v = 1, True
        continue
        
    if is_verb(actors[i]): # check if token is a verb phrase
        
        op = config.keywords[actors[i][0]] # lookup relevant keyword
        
        if not op: # ckeck if the token matches a preset token in keywords
            
            # substitute op with user specified action
            op = '=>*(%' + actors[i][0].lower() + '%)'
            
        pos_index = len(extag)
        extag += op + '(' 
        open_v = True # set current parsing of verb phrase to true
        
    elif is_conj(actors[i][0].lower()): # check if token is a conjunction
        
        op = config.keywords[actors[i][0].lower()] # lookup relevant keyword
        pos_index = len(extag)
        extag += op # append to result
        continue
        
    elif is_genitive(actors[i]): # check if token is a genitive phrase
        
        pos_index = len(extag)
        extag += actors[i][0]
        if prev_f: 
            continue
            
    # check if preposition non-keyword preposition followers an actor
    elif is_actor(actors[i][1]) and prev_p:
        
        extag = extag[:pos_index] + actors[i][0] + extag[pos_index:] # switch order of actor and preposition
        prev_p = False # set current parsing of preposition to false
        
    elif is_prep(actors[i][1]) or actors[i][0].lower() in config.keywords: # check if token is a verb phrase
        
        if open_v: 
            extag += ')' # close open verb tag
            open_v = False
        
        # substitute with keyword representation encoded in domain knowledge
        op = config.keywords[actors[i][0].lower()]
        
        if not op:
            op = '=>*(%' + actors[i][0].lower() + '%)' # substitute with user-specified token if not in keywords
            
        if op in config.filters:
            
            if flag == 1:
                
                # concatenate keyword representation to extag
                extag = extag[:pos_index] + op + '(' + extag[pos_index:]
                pos_index += len(op) + 1
                
            else:
                
                pos_index = len(extag)
                extag += op + '('
                
            prev_p, prev_f = False, True # not checking a prepositition but are checking a filter
            continue
            
        extag = extag[:pos_index] + op + extag[pos_index:] # update result
        prev_p = True # set parsing of preposition to true
        
    else:
        
        if flag == 1: # flag marks prepositional clauses succeeding a verb phrase that it modifies
            extag = extag[:pos_index] + actors[i][0] + ')' + extag[pos_index:]
            prev_f = False
            
        pos_index = len(extag)
        
        if flag == 1:
            
            flag = -1
            
            if (i+1) < len(actors) and is_actor(actors[i+1][1]):
                extag += '=>*(%filter%)(' # update result to accommodate parsed actions that require a parameter
                prev_f = True # set parsing of filter to true
            continue
            
        extag += actors[i][0] # update result
        
        if prev_f and (i+1) < len(actors) and actors[i+1][0].lower() == 'of':
            
            flag = 0
            extag += ')'
            pos_index = len(extag)
            continue
            
    if (i+1) < len(actors) and is_conj(actors[i+1][0]): # ignore conjunctions that were handled above
        continue
        
    if prev_f: # check if modifiying a filtering substitution
        
        pos_index = len(extag)
        
        if ((i+1) < len(actors) and 
            (is_genitive(actors[i+1]) or is_actor(actors[i+1][1]))):
            
            if is_actor(actors[i+1][1]):
                extag += ')=>*(%filter%)(' # accommodate action that requires a parameter
            continue
            
        extag += ')'
        
    prev_f = False
    
if open_v or prev_f:
    extag += ')'
    
for s in config.subs: # substitute tokens in the parsed result that match tokens in subs
    extag = extag.replace(s, config.subs[s])
    
extag

'(lefty batters)->(on-base percentage)=>*(%filter%)((oriole pitchers))=>*(%filter%)((fastballs)OR(curveballs))=>*(%filter%)((start velocity))=>*(%over%)((90))=>*(%filter%)((pitch))=>*(%filter%)((horizontal movement))=>*(%under%)((12))=>*(%filter%)((pitch))=>*(%filter%)((height))=>*(%over%)((2))'

In [25]:
f_list, e_list = [], [] # initialize filters, entities lists
f_keys = [re.findall(r'\(%(.+?)%\)', f_key)[0] for f_key in config.filters] # extract only content
f_keys = '(?:' + '|'.join(f_keys) + ')'
pat = r'=>\*\(' + '%' + f_keys + '%' + r'\)\(\(.+?\)\)\)?' # pattern match against any filter in domain knowledge
pat_e = r'(' + pat + ')'
f_list.append(re.findall(pat, extag))

e_list = re.split(pat_e, extag)
f_list = list(itertools.chain.from_iterable(f_list))
e_list = [expr for expr in e_list if expr and re.search(pat_e, expr) is None]
e_list = [ent for expr in e_list for ent in re.split(r'(\(.+?\)\)?)', expr) if ent] # holds non-filter entities
pos_s, pos_e = -1, -1

for i in range(len(e_list)):
    
    if e_list[i] == '=>*': # mark actions
        pos_s = i    
    elif re.search(r'\(\(.+\)\)', e_list[i]):
        pos_e = i    
    if pos_s > 0 and pos_e > 0:
        e_list[pos_s:(pos_e+1)] = [''.join(e_list[pos_s:(pos_e+1)])]
        pos_s, pos_e = -1, -1
        
print('filtered: ', end='\t'); print(f_list)
print('entity: ', end='\t'); print(e_list)

filtered: 	['=>*(%filter%)((oriole pitchers))', '=>*(%filter%)((fastballs)OR(curveballs))', '=>*(%filter%)((start velocity))', '=>*(%over%)((90))', '=>*(%filter%)((pitch))', '=>*(%filter%)((horizontal movement))', '=>*(%under%)((12))', '=>*(%filter%)((pitch))', '=>*(%filter%)((height))', '=>*(%over%)((2))']
entity: 	['(lefty batters)', '->', '(on-base percentage)']


In [28]:
config.filtered, b_feat = sequential_filter(f_list) # apply sequential filter
sequential_entity(e_list, b_feat, actors) # apply sequential entity filter


feature association: most relevant feature is team_id_pitcher
(team_id_pitcher, oriole pitchers): match_arg_to_feature_value
generate_criteria: ['balmlb']

feature association: most relevant feature is pitch_type
(pitch_type, fastballs): match_arg_to_feature_value
generate_criteria: ['FA', 'FF', 'FT', 'FC', 'FS']
(pitch_type, curveballs): match_arg_to_feature_value
generate_criteria: ['CB', 'CU']

feature association: most relevant feature is start_speed

feature association: most relevant feature is pfx_x

feature association: most relevant feature is pz
(team_id_pitcher, ['balmlb']): filter_

config.filtered = config.filtered[(config.filtered['team_id_pitcher'] == 'balmlb')]
(pitch_type, ['FA+FF+FT+FC+FS+CB+CU']): filter_

config.filtered = config.filtered[(config.filtered['pitch_type'] == 'FA') | (config.filtered['pitch_type'] == 'FF') | (config.filtered['pitch_type'] == 'FT') | (config.filtered['pitch_type'] == 'FC') | (config.filtered['pitch_type'] == 'FS') | (config.filtered['pi

**
Oracle's response: 0.3385918784691791**

**
(Relevance Feedback) Investigate Features More Like This: ['pz', 'pfx_x', 'start_speed', 'pitch_type', 'team_id_pitcher']**


In [19]:
def train_RankSVM(plotDF):
    
    '''Train a RankSVM to learn orderings between automated visualizations on the basis
    of contextual relevance (query specific) and plot aesthetics.
    
    In the ranking setting, training data constructed from a list of items 
    are constructed with an order specified between items in each list. 
    
    The order is induced by providing either a numerical/ordinal score or relevant/nonrelevant
    judgement to each item in the list.
    
    Results evaluated by Kendall Tao correlation coefficient when provided list of 
    relevant/nonrelevant visualizations.
    '''
    
    from sklearn import cross_validation
    
    X = plotDF.as_matrix() # convert pandas dataframe to numpy matrix representation
    y = plotDF[:,-1] # class labels
    X = X[:,:-1] # feature matrix
    cv = cross_validation.StratifiedShuffleSplit(y, test_size=0.2) # cross validated shuffle split
    train, test = next(iter(cv)) # construct train/test partitions
    X_train, y_train = X[train], y[train]
    X_test, y_test = X[test], y[test]
    comb = itertools.combinations(range(X_train.shape[0]), 2)
    k = 0
    Xp, yp, diff = [], [], []
    
    for (i, j) in comb:
        
        if y_train[i] == y_train[j]:
            # skip if same target or different group
            continue
        
        Xp.append(X_train[i] - X_train[j]) # compute pairwise differences across training set
        diff.append(y_train[i] - y_train[j]) # append to diff list
        yp.append(np.sign(diff[-1]))
        
        # output balanced classes
        if yp[-1] != (-1) ** k:
            yp[-1] *= -1
            Xp[-1] *= -1
            diff[-1] *= -1
        k += 1
        
    Xp, yp, diff = map(np.asanyarray, (Xp, yp, diff))
    clf = svm.SVC(kernel='linear', C=.1) # construct linear SVM to generated classify pairwise orderings
    clf.fit(Xp, yp) # fit RankSVM
    coef = clf.coef_.ravel() / np.linalg.norm(clf.coef_)
    tau, _ = scipy.stats.kendalltau(np.dot(X_test, coef), y_test) # evaluate by Kendall Tao correlation
    print('Kendall correlation coefficient: %.5f' % (tau))
    
    model_filename = os.path.join(os.path.expanduser('~'), 'Jupyter', 'Extract', 'ranksvm.pkl')
    joblib.dump(clf, model_filename) # save model
    
def test_RankSVM(plotDF):
    
    '''Return index orderings of automated visualizations encoded into plotDF'''
    
    X = plotDF.as_matrix() # convert plotDF from pandas dataframe to numpy matrix
    model_filename = os.path.join(os.path.expanduser('~'), 'Jupyter', 'Extract', 'ranksvm.pkl')
    clf = joblib.load(model_filename) # load saved model
    coef = clf.coef_.ravel() / np.linalg.norm(clf.coef_) # apply trained RankSVM to predict ranking order
    result = np.dot(X, coef)
    result = [i[0] for i in sorted(enumerate(result), key=lambda x:x[1])]
    
    return result